In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# improting required libraries for exploratory data analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv("../input/us-airbnb-open-data/AB_US_2020.csv")
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df1=df.select_dtypes('float')

# finding inifite values in dataset

In [ ]:
np.isinf(df1.stack()).groupby(level=1).sum()

In [ ]:
df1.isna().sum()

# coverting float to int

In [ ]:
df2=df.select_dtypes("int")
np.isinf(df2.stack()).groupby(level=1).sum()

In [ ]:
df2.isna().sum()

In [ ]:
df.copy()

# handling missing values 

In [ ]:
df.dropna()

In [ ]:
df.isna().sum().sum()

In [ ]:
df.isna().sum()

In [ ]:
df=df.replace(np.nan,0)

In [ ]:
df.info()

# exploratory data analysis

In [ ]:
mean_price = df.groupby('city').agg({'price': 'mean'})
mean_price

In [ ]:
sns.countplot("neighbourhood_group",data=df)
plt.xticks(rotation=90)

In [ ]:
df['neighbourhood'].describe()

In [ ]:
df['last_review'] = pd.to_datetime(df['last_review'])
df['last_review'] = pd.to_numeric(df['last_review'])
df['price'].corr(df['last_review']) 

In [ ]:
df = df.drop(['id','name','host_id','host_name','latitude','longitude','last_review'],axis=1)

In [ ]:
df.shape

# encoding labels into numeric values

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [ ]:
df['neighbourhood_group'] = df['neighbourhood_group'].replace(0,'null')
neighbourhood_group = pd.DataFrame({'Neighbourhood_group':df['neighbourhood_group'].unique()})
labelled = le.fit_transform(neighbourhood_group['Neighbourhood_group'])
neighbourhood_group['labelled'] = labelled
neighbourhood_group

In [ ]:
df["neighbourhood_group"]=le.fit_transform(df["neighbourhood_group"])
df["neighbourhood"]=le.fit_transform(df["neighbourhood"])
df["room_type"]=le.fit_transform(df["room_type"])
df["city"]=le.fit_transform(df["city"])

In [ ]:
df.info()

In [ ]:
df['minimum_nights'] = df['minimum_nights'].astype(int)
df['reviews_per_month'] = df['reviews_per_month'].astype(int)

# normalizing dataset

In [ ]:
def normalise(feature):
    nmx = 100
    nmn = 0
    
    mx = feature.max()
    mn = feature.min()
    
    return ((nmx-nmn) / (mx-mn) * (feature-mx) + nmx)

df = normalise(df)


# split dataset

# train model 

In [ ]:
x=df.drop(columns="price",axis=1)
y=df["price"]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

# linear regression model

In [ ]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
y_pred

In [ ]:
predictions = pd.DataFrame({"original_Price": y_test.values, "predicted_Price": y_pred})
predictions.head(10)

In [ ]:
sns.lmplot(x="original_Price",y="predicted_Price",data=predictions)

In [ ]:
model.score(x_test,y_test)

In [ ]:
error=y_pred-y_test
error

In [ ]:
from sklearn import metrics
model.intercept_

# Random forest regressor model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
Model=RandomForestRegressor(n_estimators=200)
Model.fit(x_train,y_train)
y_pred=Model.predict(x_test)


In [ ]:
y_pred

In [ ]:
predictions = pd.DataFrame({"original_Price": y_test.values, "predicted_Price": y_pred})
predictions.head(10)

In [ ]:
sns.scatterplot(x="original_Price",y="predicted_Price",data=predictions)